In [13]:
import PIL,torch
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
import os
import time
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from torchvision.models import resnet50


In [14]:
class SkinDataset(Dataset):
    folds=5
    def __init__(self, root , num_classes,fold=0,training=False,transform=None):
        self.data_path = []
        self.transform = transform
        if self.transform is None:
            self.transform = transforms.Compose([
                transforms.Resize(size=(224, 224)),
                transforms.ToTensor(),
            ]
            )
        for label in range(num_classes):
            self.data_dir = os.path.join(root,os.listdir(root)[label])
            self.filename = os.listdir(self.data_dir)
            l = len(self.filename)
            inter = l//SkinDataset.folds
            picked = list(range(inter* fold,inter * (fold+1))) if not training else list(range(0,inter*fold))+list(range(inter*(fold+1),l))

            for i in picked:
                file_path = os.path.join(self.data_dir , self.filename[i])
#                 img = Image.open(fil)
                self.data_path.append((file_path, label))
    
    def __getitem__(self , index):
        ddir , label = self.data_path[index]
        img = Image.open(ddir)
        result = (self.transform(img) , label)
        del img
        return result
    
    def __len__(self):
        return len(self.data_path)

In [18]:
transform = transforms.Compose([
#    transforms.RandomCrop,
#    transforms.Resize(size=(224, 224)),
    transforms.RandomResizedCrop(size=(224, 224)),
    transforms.ToTensor()
])


training_dataset = SkinDataset(os.path.join(os.getcwd(),'Skin40'), 40, training=True,transform=transform)
test_dataset = SkinDataset(os.path.join(os.getcwd(),'Skin40'), 40, training=True,transform=transform)
training_dataloader = DataLoader(training_dataset , batch_size = 128 , num_workers = 1, shuffle = True)
test_dataloader = DataLoader(test_dataset , batch_size = 128 , num_workers = 1, shuffle = True)


pop_mean = []
pop_std0 = []
pop_std1 = []
for i, (images, targets) in enumerate(training_dataloader):
    # shape (batch_size, 3, height, width)
    numpy_image = images.numpy()
    
    # shape (3,)
    batch_mean = np.mean(numpy_image, axis=(0,2,3))
    batch_std0 = np.std(numpy_image, axis=(0,2,3))
    batch_std1 = np.std(numpy_image, axis=(0,2,3), ddof=1)
    
    pop_mean.append(batch_mean)
    pop_std0.append(batch_std0)
    pop_std1.append(batch_std1)
    
for i, (images, targets) in enumerate(test_dataloader):
    # shape (batch_size, 3, height, width)
    numpy_image = images.numpy()
    
    # shape (3,)
    batch_mean = np.mean(numpy_image, axis=(0,2,3))
    batch_std0 = np.std(numpy_image, axis=(0,2,3))
    batch_std1 = np.std(numpy_image, axis=(0,2,3), ddof=1)
    
    pop_mean.append(batch_mean)
    pop_std0.append(batch_std0)
    pop_std1.append(batch_std1)

# shape (num_iterations, 3) -> (mean across 0th axis) -> shape (3,)
pop_mean = np.array(pop_mean).mean(axis=0)
pop_std0 = np.array(pop_std0).mean(axis=0)
pop_std1 = np.array(pop_std1).mean(axis=0)

# pop_mean , pop_std 在transforms.normalize()中直接使用
print(pop_mean)
print(pop_std0)
print(pop_std1)

[0.64233315 0.49493808 0.4587943 ]
[0.23893961 0.21936509 0.21956368]
[0.23893966 0.2193651  0.21956368]


In [19]:
transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor()
])


training_dataset = SkinDataset(os.path.join(os.getcwd(),'Skin40'), 40, training=True,transform=transform)
test_dataset = SkinDataset(os.path.join(os.getcwd(),'Skin40'), 40, training=True,transform=transform)
training_dataloader = DataLoader(training_dataset , batch_size = 128 , num_workers = 1, shuffle = True)
test_dataloader = DataLoader(test_dataset , batch_size = 128 , num_workers = 1, shuffle = True)


pop_mean = []
pop_std0 = []
pop_std1 = []
for i, (images, targets) in enumerate(training_dataloader):
    # shape (batch_size, 3, height, width)
    numpy_image = images.numpy()
    
    # shape (3,)
    batch_mean = np.mean(numpy_image, axis=(0,2,3))
    batch_std0 = np.std(numpy_image, axis=(0,2,3))
    batch_std1 = np.std(numpy_image, axis=(0,2,3), ddof=1)
    
    pop_mean.append(batch_mean)
    pop_std0.append(batch_std0)
    pop_std1.append(batch_std1)
    
for i, (images, targets) in enumerate(test_dataloader):
    # shape (batch_size, 3, height, width)
    numpy_image = images.numpy()
    
    # shape (3,)
    batch_mean = np.mean(numpy_image, axis=(0,2,3))
    batch_std0 = np.std(numpy_image, axis=(0,2,3))
    batch_std1 = np.std(numpy_image, axis=(0,2,3), ddof=1)
    
    pop_mean.append(batch_mean)
    pop_std0.append(batch_std0)
    pop_std1.append(batch_std1)

# shape (num_iterations, 3) -> (mean across 0th axis) -> shape (3,)
pop_mean = np.array(pop_mean).mean(axis=0)
pop_std0 = np.array(pop_std0).mean(axis=0)
pop_std1 = np.array(pop_std1).mean(axis=0)

# pop_mean , pop_std 在transforms.normalize()中直接使用
print(pop_mean)
print(pop_std0)
print(pop_std1)

[0.6076746  0.49058664 0.45970112]
[0.2578426  0.2387751  0.24144615]
[0.2578426  0.23877512 0.24144617]
